In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [2]:
# Load datasets
users = pd.read_csv('synthetic_users.csv')
products = pd.read_csv('synthetic_products.csv')
interactions = pd.read_csv('synthetic_interactions.csv') 

In [12]:
interaction_type_scores = {
    'view': 1,
    'comment': 2,
    'like': 3,
    'share': 4,
    'purchase': 5
}

# Convert interaction types to scores
interactions['interaction'] = interactions['interaction_type'].map(interaction_type_scores)

In [13]:
# Function to preprocess products data
def preprocess_products(products):
    # Normalize numerical features
    scaler = MinMaxScaler()
    products[['price', 'rating']] = scaler.fit_transform(products[['price', 'rating']])

    # Vectorize product descriptions using TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
    product_descriptions_matrix = vectorizer.fit_transform(products['description'])

    # Combine product metadata into a single feature set
    product_features = np.hstack((products[['price', 'rating']], product_descriptions_matrix.toarray()))

    # Separate and manage product categories
    product_categories = products[['product_id', 'category']]
    products = products.drop(columns=['category'])

    return products, product_features, product_categories

In [14]:
# Preprocess products
products, product_features, product_categories = preprocess_products(products)

# Aggregate the interactions by taking the average score for each user-product pair
aggregated_interactions = interactions.groupby(['user_id', 'product_id']).mean().reset_index()

# Create user-item interaction matrix
user_item_matrix = aggregated_interactions.pivot(index='user_id', columns='product_id', values='interaction').fillna(0)

# Convert to sparse matrix format
user_item_sparse = csr_matrix(user_item_matrix.values)

TypeError: agg function failed [how->mean,dtype->object]